In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import torch
from transformers.file_utils import is_tf_available, is_torch_available
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [3]:
data_news = pd.read_csv('fake_or_real_news.csv')
print(data_news.shape)
data_news[:15]

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [4]:
# This code segment parses the data_news dataset into a more manageable format

titles = []
tokenized_titles = []
sequence_labels = data_news['label']

title, tokenized_title =  [], []
for news in data_news['title']:
    title.append(news)
    tokenized_title.append(news.split(' '))

In [5]:
# Python list for each news
title[0], tokenized_title[0], sequence_labels[0]

('You Can Smell Hillary’s Fear',
 ['You', 'Can', 'Smell', 'Hillary’s', 'Fear'],
 'FAKE')

In [6]:
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['FAKE', 'REAL']

In [7]:
# Convert sequence_labels to indices:
label_indices = [unique_sequence_labels.index(l) for l in sequence_labels]

news_dataset = Dataset.from_dict(
    dict(
        titles=title, 
        label=label_indices,
        tokens=tokenized_title,
    )
)
news_dataset = news_dataset.train_test_split(test_size=0.2)

news_dataset['train'][0]


{'titles': 'Get Ready For A Fight To Replace Scalia',
 'label': 1,
 'tokens': ['Get', 'Ready', 'For', 'A', 'Fight', 'To', 'Replace', 'Scalia']}

In [33]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base-clf')


loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


In [9]:
def preprocess_function(examples):
    return tokenizer(examples["titles"], truncation=True, padding=True)# truncation=True makes sure to exludes instances with more 512 tokens

In [10]:
# go over all our data set, tokenize them
seq_clf_tokenized_news = news_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5068 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [11]:
seq_clf_tokenized_news['train'][0]


{'titles': 'Get Ready For A Fight To Replace Scalia',
 'label': 1,
 'tokens': ['Get', 'Ready', 'For', 'A', 'Fight', 'To', 'Replace', 'Scalia'],
 'input_ids': [0,
  14181,
  19325,
  286,
  83,
  14381,
  598,
  42439,
  31462,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [13]:
sequence_clf_model = RobertaForSequenceClassification.from_pretrained('roberta-base', 
                                                                         num_labels=len(unique_sequence_labels),)

# set an index -> label dictionary
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
sequence_clf_model.config


RobertaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "FAKE",
    "1": "REAL"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):  # common method to take in logits and calculate accuracy of the eval set
    logits, labels = eval_pred   # logit and label are returning from training loop
    predictions = np.argmax(logits, axis=-1) 
    return metric.compute(predictions=predictions, references=labels) # compute the accuracy

metric = evaluate.load("accuracy")
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#####################################################

def compute_metrics_binary(eval_pred):
    """metrics for binary classification"""
    
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # Calculate the AUC score
    auc_score = roc_auc_score(labels, preds)

    # Calculate the accuracy, true positive, false positive, false negative, and true negative values
    acc = metric.compute(predictions=preds, references=labels)
    tp = ((preds >= 0.5) & (labels == 1)).sum()
    fp = ((preds >= 0.5) & (labels == 0)).sum()
    fn = ((preds < 0.5) & (labels == 1)).sum()
    tn = ((preds < 0.5) & (labels == 0)).sum()

    # Calculate the precision, recall, and F1 score
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return {
        'Validation Accuracy': acc['accuracy'],
        'Validation Precision': auc_score,
        'Validation AUC': precision,
        'Validation Recall': recall,
        'Validation F1_Score': f1_score,
        'Validation TP': tp,
        'Validation FP': fp,
        'Validation FN': fn,
        'Validation TN': tn,
    }

#####################################################

from sklearn.metrics import classification_report

def compute_metrics_multiclass(eval_pred):
    """metrics for multiclass classification"""
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    report = classification_report(labels, preds, output_dict=True)
    acc_score = report['accuracy']
    pre_score = report['macro avg']['precision']
    rcl_score = report['macro avg']['recall']
    f1_score = report['macro avg']['f1-score']

    return {
        'Validation Accuracy': acc_score,
        'Validation Macro Recall': rcl_score,
        'Validation Macro Precision': pre_score,        
        'Validation Macro F1_Score': f1_score,
        }

In [ ]:
epochs = 10

# Training argument
training_args = TrainingArguments(
    output_dir="./roberta_news_clf/results", # Local directory to save check point of our model as fitting
    num_train_epochs=epochs,         # minimum of two epochs
    per_device_train_batch_size=32,  # batch size for training and evaluation, it common to take around 32, 
    per_device_eval_batch_size=32,   # sometimes less or more, The smaller batch size, the more change model update 
    load_best_model_at_end=True,     # Even if we overfit the model by accident, load the best model through checkpoint
    
    # some deep learning parameters that the trainer is able to take in
    warmup_steps = len(seq_clf_tokenized_news['train']) // 5,  # learning rate scheduler by number of warmup steps
    weight_decay = 0.05,    # weight decay for our learning rate schedule (regularization)
    
    logging_steps = 1,  # Tell the model minimum number of steps to log between (1 means logging as much as possible)
    log_level = 'info',
    eval_strategy = 'epoch', # It is "steps" or "epoch", we choose epoch: how many times to stop training to test
    eval_steps = 50,
    save_strategy = 'epoch',  # save a check point of our model after each epoch
    learning_rate=2e-5,  # learning rate for our model
)

# Define the trainer:
trainer = Trainer(
    model=sequence_clf_model,   # take our model (sequence_clf_model)
    args=training_args,         # we just set it above
    train_dataset=seq_clf_tokenized_news['train'], # training part of dataset
    eval_dataset=seq_clf_tokenized_news['test'],   # test (evaluation) part of dataset
    compute_metrics=compute_metrics_binary,    # This part is optional but we want to calculate accuracy of our model 
    data_collator=data_collator         # data colladior with padding. Infact, we may or may not need a data collator
                                        # we can check the model to see how it lookes like with or without the collator
)

In [17]:
# Get initial metrics: evaluation on test set
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tokens, titles. If tokens, titles are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1267
  Batch size = 32
c:\Users\quang\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


C:\Users\quang\AppData\Local\Temp\ipykernel_30164\3547021378.py:39: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


{'eval_loss': 0.7036638855934143,
 'eval_model_preparation_time': 0.003,
 'eval_Validation Accuracy': 0.47829518547750594,
 'eval_Validation Precision': 0.5,
 'eval_Validation AUC': nan,
 'eval_Validation Recall': 0.0,
 'eval_Validation F1_Score': nan,
 'eval_Validation TP': 0,
 'eval_Validation FP': 0,
 'eval_Validation FN': 661,
 'eval_Validation TN': 606,
 'eval_runtime': 33.8481,
 'eval_samples_per_second': 37.432,
 'eval_steps_per_second': 1.182}

In [18]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tokens, titles. If tokens, titles are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,068
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1,590
  Number of trainable parameters = 124,647,170
c:\Users\quang\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Model Preparation Time,Validation accuracy,Validation precision,Validation auc,Validation recall,Validation f1 Score,Validation tp,Validation fp,Validation fn,Validation tn
1,0.192200,0.317413,0.003000,0.865036,0.864403,0.864583,0.878971,0.871718,581,91,80,515
2,0.705300,0.280540,0.003000,0.891871,0.893142,0.923948,0.863843,0.892885,571,47,90,559
3,0.024500,0.272720,0.003000,0.899763,0.898579,0.886957,0.925870,0.905996,612,78,49,528
4,0.031200,0.305627,0.003000,0.900552,0.900365,0.904690,0.904690,0.904690,598,63,63,543
5,0.029500,0.355812,0.003000,0.905288,0.904148,0.892598,0.930408,0.911111,615,74,46,532
6,0.003300,0.383048,0.003000,0.892660,0.890260,0.862069,0.945537,0.901876,625,100,36,506
7,0.593600,0.629158,0.003000,0.841358,0.834982,0.774463,0.981846,0.865911,649,189,12,417
8,0.000500,0.595834,0.003000,0.903710,0.901811,0.879044,0.945537,0.911079,625,86,36,520
9,0.001900,0.601822,0.003000,0.902920,0.901467,0.885387,0.934947,0.909492,618,80,43,526
10,0.000500,0.620531,0.003000,0.907656,0.906349,0.891931,0.936460,0.913653,619,75,42,531


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tokens, titles. If tokens, titles are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1267
  Batch size = 32
Saving model checkpoint to ./roberta_news_clf/results\checkpoint-159
Configuration saved in ./roberta_news_clf/results\checkpoint-159\config.json
Model weights saved in ./roberta_news_clf/results\checkpoint-159\model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in ./roberta_news_clf/results\checkpoint-159\tokenizer_config.json
Special tokens file saved in ./roberta_news_clf/results\checkpoint-159\special_tokens_map.json
c:\Users\quang\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no a

TrainOutput(global_step=1590, training_loss=0.18001216948024143, metrics={'train_runtime': 10981.5302, 'train_samples_per_second': 4.615, 'train_steps_per_second': 0.145, 'total_flos': 2421045153666720.0, 'train_loss': 0.18001216948024143, 'epoch': 10.0})

In [19]:
trainer.evaluate()


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tokens, titles. If tokens, titles are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1267
  Batch size = 32
c:\Users\quang\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.2727195918560028,
 'eval_model_preparation_time': 0.003,
 'eval_Validation Accuracy': 0.8997632202052092,
 'eval_Validation Precision': 0.89857851140636,
 'eval_Validation AUC': 0.8869565217391304,
 'eval_Validation Recall': 0.9258698940998488,
 'eval_Validation F1_Score': 0.9059955588452998,
 'eval_Validation TP': 612,
 'eval_Validation FP': 78,
 'eval_Validation FN': 49,
 'eval_Validation TN': 528,
 'eval_runtime': 37.6921,
 'eval_samples_per_second': 33.614,
 'eval_steps_per_second': 1.061,
 'epoch': 10.0}

In [32]:
trainer.save_model()
model_path = "./roberta-base-clf"
sequence_clf_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Saving model checkpoint to ./roberta_news_clf/results
Configuration saved in ./roberta_news_clf/results\config.json
Model weights saved in ./roberta_news_clf/results\model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in ./roberta_news_clf/results\tokenizer_config.json
Special tokens file saved in ./roberta_news_clf/results\special_tokens_map.json
Configuration saved in ./roberta-base-clf\config.json
Model weights saved in ./roberta-base-clf\model.safetensors
tokenizer config file saved in ./roberta-base-clf\tokenizer_config.json
Special tokens file saved in ./roberta-base-clf\special_tokens_map.json


('./roberta-base-clf\\tokenizer_config.json',
 './roberta-base-clf\\special_tokens_map.json',
 './roberta-base-clf\\vocab.json',
 './roberta-base-clf\\merges.txt',
 './roberta-base-clf\\added_tokens.json',
 './roberta-base-clf\\tokenizer.json')

In [21]:
from transformers import pipeline
pipe = pipeline("text-classification", "./roberta-base", tokenizer=tokenizer)


loading configuration file ./roberta-base\config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "FAKE",
    "1": "REAL"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": null,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file ./roberta-base\config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForSequenceClassification"
 

In [23]:
text = "skibidi toilet just died"
pipe(text)

[{'label': 'FAKE', 'score': 0.9685542583465576}]